In [1]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.ticker import MultipleLocator
from sklearn.preprocessing import LabelEncoder
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
import Common_Functions as cmn

# Combine All Regressions into one Graph

In [2]:
sns.set_context("poster")
sns.set_style("ticks")

In [3]:
observed_price = cmn.get_stringless_training_data()
prices = observed_price.iloc[:,-1:]
rf = pd.read_csv("Exported_Data/RandomForest_for_weights.csv")
prices["Random Forest Prediction"] = rf.iloc[:,-1:]

gbr = pd.read_csv("Exported_Data/GBR_for_weights.csv")
prices["Gradient Boosted Regressor Prediction"] = gbr.iloc[:,-1:]

ols = pd.read_csv("Exported_Data/OLS_for_weights.csv")
prices["OLS Prediction"] = ols.iloc[:,-1:]

xgb = pd.read_csv("Exported_Data/XGB_for_weights.csv")
prices["XGBoost Regressor"] = xgb.iloc[:,-1:]

prices.head(10)

,SalePrice,Random Forest Prediction,Gradient Boosted Regressor Prediction,OLS Prediction,XGBoost Regressor
0,208500,200024.618000,203134.857093,227243.598710,203864.280
1,181500,164336.991725,166604.447006,196355.868944,166006.050
2,223500,214510.291750,214568.225191,222831.166926,217937.500
3,140000,180331.061450,182574.606910,199358.318716,176473.830
4,250000,291077.017075,296219.484848,295552.240770,286894.940
5,143000,154814.209450,156977.535760,178645.733189,158264.860
6,307000,295568.982250,289599.131700,272538.422890,304965.160
7,200000,218701.756600,211407.271823,251870.815786,224092.600
8,129900,173462.545825,162558.878411,189753.427301,166114.940
9,118000,121473.271425,119907.948215,83918.353787,117998.414


# Weightings based on Perceived Accuracy

In [11]:
observed_price = cmn.get_stringless_training_data()
prices = observed_price.iloc[:,-1:]
rf = pd.read_csv("Exported_Data/RandomForestPrediction.csv")
prices["Random Forest Prediction"] = rf.iloc[:,-1:]

gbr = pd.read_csv("Exported_Data/GBR.csv")
prices["Gradient Boosted Regressor Prediction"] = gbr.iloc[:,-1:]

ols = pd.read_csv("Exported_Data/OLS.csv")
prices["OLS Prediction"] = ols.iloc[:,-1:]

xgb = pd.read_csv("Exported_Data/XGB.csv")
prices["XGB Regressor Prediction"] = xgb.iloc[:,-1:]

In [45]:
weight = [.3, .4, .4, .6]
columns = ["Random Forest Prediction", "Gradient Boosted Regressor Prediction", "OLS Prediction", "XGB Regressor Prediction"]
prices["Weighted Average"] = np.average(prices[columns], weights=weight, axis=1)

prices.head(10)

,SalePrice,Random Forest Prediction,Gradient Boosted Regressor Prediction,OLS Prediction,XGB Regressor Prediction,Weighted Average
0,208500,200024.618000,203134.857093,227243.598710,203864.280,208516.079836
1,181500,164336.991725,166604.447006,196355.868944,166006.050,172993.443469
2,223500,214510.291750,214568.225191,222831.166926,217937.500,217691.379042
3,140000,180331.061450,182574.606910,199358.318716,176473.830,183974.580403
4,250000,291077.017075,296219.484848,295552.240770,286894.940,291863.976100
5,143000,154814.209450,156977.535760,178645.733189,158264.860,162148.521420
6,307000,295568.982250,289599.131700,272538.422890,304965.160,292061.654418
7,200000,218701.756600,211407.271823,251870.815786,224092.600,226692.542367
8,129900,173462.545825,162558.878411,189753.427301,166114.940,172136.852960
9,118000,121473.271425,119907.948215,83918.353787,117998.414,111042.088605


In [47]:
#weighted = prices["Weighted Average"]
weighted = pd.DataFrame(i for i in range(1461,2920))
weighted = weighted.rename(columns={0:'Id'})
weighted["SalePrice"] = prices['Weighted Average']
weighted = weighted.set_index("Id")

pd.DataFrame(weighted).to_csv(f"{cmn.export_data_path}Weighted Submission.csv")